# **Análisis de las diferencias socioeconómicas en la postulación a las Universidades en Chile**: 

### **Parte: Limpieza y Tratamiento de los datos estudiantiles**






In [1]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import os
import pyproj

from json import load
from zipfile import ZipFile
from shapely.geometry import Point
from extras.ptje_ponderado import remplazar_ponderado
from extras.tipo_dep import remplazar_cod_depe


En la variable `parameters` se encuentras las columnas que se van a utilizar para el analisis de los datos. La explicacion de dichas variables se encuentra en el notebook `exploracion_analisis.ipynb` en el apartado de los **Datos**.

In [2]:
parameters = load(open(os.path.join("extras", "parametros.json"), "r"))
%mkdir -p data

In [3]:
# Descomprimimos el archivo data.zip que contiene los datos educativos

with ZipFile(os.path.join("raw_data", "data.zip")) as zp:
    zp.extractall(os.path.join("raw_data"))

In [4]:
# Lectura de los archivos

df = dict()
for index, path in enumerate(parameters["PATH"]):
    df[parameters["NOM"][index]] = pd.read_csv(os.path.join(*path), sep=";", low_memory=False)

#### **Limpieza de los datos de los estudiantes inscritos para rendir la PAES y los matriculados en el proceso de admision 2023.**

En este apartado se van a generar dos dataframes, uno con los datos de los estudiantes inscritos para rendir la PAES y otro con los datos de los estudiantes matriculados en el proceso de admision 2023.

---

En relación al dataframe `df_estudiantes`, es importante destacar que no se eliminarán las filas que presenten puntajes CLEC o MAT1 iguales a 0. Este criterio se respalda en la información obtenida por el MINEDUC, la cual está detallada en el correo adjunto en el archivo `extras/resposte.pdf`. Según dicha información, estos casos corresponden a estudiantes válidos que no asistieron a las pruebas de CLEC o MATE1.

No obstante, se ha establecido una excepción para aquellos estudiantes cuyo promedio de notas sea igual a 0 y, además, no dispongan de puntaje NEM ni ranking. En estos casos particulares, se ha decidido eliminar las filas correspondientes. Esta elección se basa en la imposibilidad de determinar la validez de la información, ya que la falta de datos fundamentales dificulta la evaluación de la situación académica de dichos estudiantes.

In [5]:
# Estudiantes inscritos para la PAES 2023

df_estudiantes = df["inscritos_puntajes_2023"].merge(df["socioeconomico_domicilio_2023"], 
                                    how="inner", on="MRUN", suffixes=(" ", " "))

# Remplazo de datos faltantes
df_estudiantes.PROMEDIO_NOTAS = df_estudiantes.PROMEDIO_NOTAS.apply(lambda x: x.replace(",", ".")).astype(float)
df_estudiantes.PROMEDIO_CM_MAX = df_estudiantes.PROMEDIO_CM_MAX.apply(lambda x: x.replace(",", ".")).astype(float)
df_estudiantes.CODIGO_COMUNA_EGRESO = df_estudiantes.CODIGO_COMUNA_EGRESO.apply(lambda x: 0 if x == " " else x).astype(int)
df_estudiantes.DEPENDENCIA = df_estudiantes.DEPENDENCIA.apply(lambda x: 0 if x == " " else x).astype(int) # Si no se sabe el tipo de educación
df_estudiantes.DEPENDENCIA = df_estudiantes.DEPENDENCIA.apply(lambda x: remplazar_cod_depe(x)).astype(str) # Remplazamos el código de dependencia por el tipo de dependencia

df_estudiantes = df_estudiantes.query("PROMEDIO_NOTAS != 0 & PTJE_NEM != 0 & PTJE_RANKING != 0") # Eliminamos los estudiantes que no poseen datos de notas
df_estudiantes.loc[:, "RBD"] = df_estudiantes.RBD.apply(lambda x: 0 if x == " " else x) # Si el estudiante no posee RBD, se le asigna "No info"
df_estudiantes.loc[:, "RBD"] = df_estudiantes.RBD.astype(int) 


df_estudiantes = df_estudiantes[parameters["VAR1"]]
df_estudiantes.to_parquet(os.path.join("data", "estudiantes_2023.parquet"), engine="pyarrow", compression="snappy")


/home/rocky17/miniconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Este código realiza varias operaciones de limpieza y transformación de datos en un DataFrame de pandas llamado `df_estudiantes`:

1. Reemplaza todas las comas por puntos en las columnas `PROMEDIO_NOTAS` y `PROMEDIO_CM_MAX`, y luego convierte los valores a tipo float. Esto se hace porque en algunos países se usa la coma como separador decimal, pero Python usa el punto.

2. En las columnas `CODIGO_COMUNA_EGRESO` y `DEPENDENCIA`, reemplaza los espacios en blanco por ceros y luego convierte los valores a tipo int. Esto se hace para manejar los datos faltantes.

3. Aplica una función `remplazar_cod_depe` a la columna `DEPENDENCIA` para reemplazar los códigos de dependencia por el tipo de dependencia, y luego convierte los valores a tipo str.

4. Filtra el DataFrame para eliminar las filas donde `PROMEDIO_NOTAS`, `PTJE_NEM` y `PTJE_RANKING` son cero. Esto se hace para eliminar los estudiantes que no tienen datos de notas.

5. En la columna `RBD`, reemplaza los espacios en blanco por ceros y luego convierte los valores a tipo int. Esto se hace para manejar los datos faltantes.

Este código realiza varias operaciones de limpieza y transformación de datos en un DataFrame de pandas llamado `df_estudiantes`:

1. Reemplaza todas las comas por puntos en las columnas `PROMEDIO_NOTAS` y `PROMEDIO_CM_MAX`, y luego convierte los valores a tipo float. Esto se hace porque en algunos países se usa la coma como separador decimal, pero Python usa el punto.

2. En las columnas `CODIGO_COMUNA_EGRESO` y `DEPENDENCIA`, reemplaza los espacios en blanco por ceros y luego convierte los valores a tipo int. Esto se hace para manejar los datos faltantes.

3. Aplica una función `remplazar_cod_depe` a la columna `DEPENDENCIA` para reemplazar los códigos de dependencia por el tipo de dependencia, y luego convierte los valores a tipo str.

4. Filtra el DataFrame para eliminar las filas donde `PROMEDIO_NOTAS`, `PTJE_NEM` y `PTJE_RANKING` son cero. Esto se hace para eliminar los estudiantes que no tienen datos de notas.

5. En la columna `RBD`, reemplaza los espacios en blanco por ceros y luego convierte los valores a tipo int. Esto se hace para manejar los datos faltantes.

Agrupamos los estudiantes por tipo de establecimiento, si este es gratuito, particular subvencionado o particular pagado o corporación municipal. Ya que 

In [6]:
df_estudiantes

,MRUN,RBD,SEXO,DEPENDENCIA,CODIGO_REGION_DOMICILIO,CODIGO_COMUNA_EGRESO,INGRESO_PERCAPITA_GRUPO_FA,CONEXION_INSUFICIENTE,PROMEDIO_NOTAS,PTJE_NEM,PTJE_RANKING,CLEC_MAX,MATE1_MAX
0,3703667,5654,1,PARTICULAR SUBVENCIONADO,9,9101,8,1,5.15,463,546,816,846
2,17372901,3645,1,GRATUITO,16,16101,5,9,6.05,731,798,498,499
3,16702090,109,2,GRATUITO,1,1101,3,9,5.53,582,582,0,0
4,22654139,3003,2,PARTICULAR SUBVENCIONADO,7,7101,5,2,6.40,830,875,364,402
6,13700650,9747,2,GRATUITO,13,13121,3,1,5.47,569,569,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296807,26526755,8518,1,GRATUITO,13,13106,1,9,5.23,488,491,633,757
296808,26526694,286,1,GRATUITO,2,2101,2,3,6.03,722,809,761,516
296809,26526332,8926,1,GRATUITO,13,13123,1,2,5.87,680,679,582,480
296810,26525532,279,1,GRATUITO,2,2101,3,2,5.95,700,703,606,579


A su vez, llevamos a cabo la sustitución de los RBD vacíos por 0. Esta elección se fundamenta en el hecho de que, en el caso de requerir el RBD, la ausencia de este dato indica que dichos estudiantes no serán considerados. No obstante, es importante señalar que si el enfoque del análisis recae en promedios de notas, puntajes y ranking, estos estudiantes sí serán tenidos en cuenta en el estudio, aún cuando el RBD esté ausente o sea igual a 0. Ademas tambien hemos remplazados las comunas y el codigo de dependencia (tipo de educacion) por 0, y su justificacion es la misma que la del RBD.

In [7]:
# Estudiantes matriculados en el proceso de admision 2023

df_matriculados= df_estudiantes.merge(df["matricula_2023"], how="inner", on="MRUN", suffixes=(" ", " "))

df_matriculados = df_matriculados[parameters["VAR2"]]



Generamos un dataframe llamado `df_matriculados`, que contiene información sobre los estudiantes matriculados con valores válidos para el análisis. En este contexto, dado que ambas pruebas son obligatorias, se presupone que el estudiante debió haberlas rendido como requisito mínimo para la postulación (a priori). Sin embargo, es importante destacar que existen vías de admisión que ya no requieren la presentación de la prueba PAES. Ante esta consideración, hemos tomado la decisión de conservar en nuestro análisis aquellos estudiantes que no participaron en la rendición de la PAES.

##### Verificación de los alumnos que no rindieron la PAES:

Notemos que al filtrar por ```CLEC_MAX == 0``` | ```MATE1_MAX == 0``` y al aplicar ```unique()```, obtenemos vias de admision distintas a 1. Lo que se trata de estudiantes que no dieron alguna de las pruebas obligatorias, pero que accedieron por vias de admision especial (no via PAES).


In [8]:
df_matriculados.query("CLEC_MAX == 0 | MATE1_MAX == 0").VIA_ADMISION.unique()

array([ 7,  9,  8,  2, 10])

A su vez, tambien agregamos los puntajes ponderados de los estudiantes que accedieron **via PAES**, o por la via de admision 1.

In [9]:
df_matriculados = remplazar_ponderado(df_matriculados, df["puntajes_seleccion_2023"])

df_matriculados.fillna(0, inplace=True)
col_move = df_matriculados.pop("PTJE_PONDERADO")
df_matriculados.insert(12, "PTJE_PONDERADO", col_move)
df_matriculados.drop(columns=["CODIGO_CARRERA"], inplace=True)

df_matriculados.to_parquet(os.path.join("data", "matriculados_2023.parquet"), engine="pyarrow", compression="snappy")

/home/rocky17/miniconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [10]:
df_matriculados

,MRUN,SEXO,DEPENDENCIA,CODIGO_REGION_DOMICILIO,CODIGO_COMUNA_EGRESO,INGRESO_PERCAPITA_GRUPO_FA,PROMEDIO_NOTAS,PTJE_NEM,PTJE_RANKING,CLEC_MAX,MATE1_MAX,PTJE_PONDERADO,NOMBRE_INSTITUCION,VIA_ADMISION
0,17433853,2,GRATUITO,13,15101,99,5.15,463,466,674,499,0.00,UNIVERSIDAD DIEGO PORTALES,7
1,23855103,1,NO INFO,2,0,99,5.18,473,483,742,499,610.30,UNIVERSIDAD DE PLAYA ANCHA,1
2,19427243,2,NO INFO,13,0,7,5.13,456,465,663,499,571.60,UNIVERSIDAD CATOLICA SILVA HENRIQUEZ,1
3,7516963,2,PARTICULAR SUBVENCIONADO,13,5502,5,4.48,252,252,564,437,0.00,UNIVERSIDAD BERNARDO O'HIGGINS,7
4,20896830,1,NO INFO,9,0,4,5.00,415,425,692,462,513.60,UNIVERSIDAD DE LA FRONTERA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123678,27125516,2,PARTICULAR SUBVENCIONADO,10,10301,2,5.85,672,704,551,480,591.45,UNIVERSIDAD DE LOS LAGOS,1
123679,26525702,2,PARTICULAR SUBVENCIONADO,10,10101,5,5.90,685,713,617,452,580.50,UNIVERSIDAD SAN SEBASTIAN,1
123680,26526984,1,GRATUITO,10,10301,8,6.23,780,839,729,507,0.00,UNIVERSIDAD DE LOS LAGOS,3
123681,26526755,1,GRATUITO,13,13106,1,5.23,488,491,633,757,593.40,UNIVERSIDAD CENTRAL,1


#### **Liempza de los datos sobre los establecimientos educacionales**

Se creara un `dataframe` que contenga los promedios de las pruebas CLEC y MAT1 por establecimiento educativo, ademas de la cantidad de estudiantes que rindieron la prueba en dicho establecimiento. Para ello se va usar el `RBD` como identificador del establecimiento educativo, y se le realizara un `pd.merge` con el `dataframe` de los establecimientos.

En `mask_df` destarmaos los estudiantes que no tengan RBD y COD_COMUNA_RBD, ya que no podemos identificar el establecimiento educativo al que pertenecen.

In [11]:
mask_df = (df_estudiantes.RBD != 0) & ((df_estudiantes.CLEC_MAX != 0) | (df_estudiantes.MATE1_MAX != 0)) & (df_estudiantes.CODIGO_COMUNA_EGRESO != 0) 

df_promedios = df_estudiantes[mask_df].groupby("RBD").agg({
    "PROMEDIO_NOTAS": "mean",
    "PTJE_NEM": "mean",
    "PTJE_RANKING": "mean",
    "CLEC_MAX": "mean",
    "MATE1_MAX": "mean"})

df_promedios["ALUMNOS"] = df_estudiantes[mask_df].groupby("RBD").size()

La finalidad de este `dataframe` es para poder agregar los promedios de `notas`, `puntaje NEM`, `ranking`, `PAES de compresion lectora(CLEC)` y `PAES de matematicas (MATE1)` por establecimiento educativo.

In [12]:
mat = df["matricula_EE_2022"][["RBD", "COD_DEPE", "COD_REG_RBD", "COD_COM_RBD", "MAT_TOTAL"]]
doc = df["dotacion_docente_2023"][["RBD", "DC_TOT"]]
geo = df["directorio_EE_2022"][["RBD", "LATITUD", "LONGITUD"]]

geo = geo.query("LATITUD != {a} & LONGITUD != {a}".format(a="' '")) # Eliminar los valores vacios

df_establecimientos = mat.merge(doc, how="inner", on="RBD", suffixes=(" ", " ")).merge(geo, how="inner", on="RBD", suffixes=(" ", " "))
df_establecimientos = df_establecimientos.merge(df_promedios, how="inner", on="RBD", suffixes=(" ", " "))

df_establecimientos.LATITUD = df_establecimientos.LATITUD.str.replace(",", ".").astype(float)
df_establecimientos.LONGITUD = df_establecimientos.LONGITUD.str.replace(",", ".").astype(float)

df_establecimientos["geometry"] = df_establecimientos.apply(lambda x: Point(x.LONGITUD, x.LATITUD), axis=1)


Finalmente, de los 290,000 estudiantes, se incluyó la suma de 261,000 que participaron en una o ambas pruebas obligatorias. Estos resultados se incorporaron a sus respectivos establecimientos educativos. En otras palabras, basándonos en el RBD del alumno, se añadió el promedio de notas de CLEC y MAT1, junto con la cantidad de estudiantes que realizaron la prueba en ese establecimiento.

In [13]:
df_establecimientos.ALUMNOS.sum()

261088

Esto es mas que nada para verificar que no se perdió información en el proceso de limpieza de los datos, y asegurarnos de poseer una buena cantidad de datos para el análisis.

Ahora lo pasamos a un `geopandas`:

In [14]:
df_establecimientos.drop(["LATITUD", "LONGITUD"], axis=1, inplace=True)

gpd_establecimientos = gpd.GeoDataFrame(df_establecimientos, geometry="geometry")
gpd_establecimientos.COD_DEPE = gpd_establecimientos.COD_DEPE.apply(lambda x: remplazar_cod_depe(x)).astype(str)
gpd_establecimientos.crs = pyproj.CRS.from_epsg(5360)
gpd_establecimientos.to_parquet(os.path.join("data", "establecimientos_2023.parquet"), engine="pyarrow", compression="snappy")

/home/rocky17/miniconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [15]:
gpd_establecimientos

,RBD,COD_DEPE,COD_REG_RBD,COD_COM_RBD,MAT_TOTAL,DC_TOT,PROMEDIO_NOTAS,PTJE_NEM,PTJE_RANKING,CLEC_MAX,MATE1_MAX,ALUMNOS,geometry
0,1,GRATUITO,15,15101,667,70,5.855690,672.137931,708.396552,517.000000,488.620690,58,POINT (-70.29521 -18.48720)
1,3,GRATUITO,15,15101,376,28,5.200000,483.000000,490.000000,465.000000,0.000000,1,POINT (-70.26083 -18.50358)
2,4,GRATUITO,15,15101,1014,63,6.259268,787.540845,790.873239,642.988732,555.343662,355,POINT (-70.30827 -18.47424)
3,5,GRATUITO,15,15101,771,66,6.165469,760.453125,781.046875,617.807292,535.395833,192,POINT (-70.31345 -18.47690)
4,7,GRATUITO,15,15101,1163,99,5.864043,675.127660,685.051064,535.510638,469.229787,235,POINT (-70.28837 -18.47615)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3599,41702,PARTICULAR,8,8112,770,28,6.055270,728.459459,737.094595,694.202703,602.675676,74,POINT (-73.10917 -36.79871)
3600,41780,PARTICULAR,3,3101,356,19,5.835556,664.962963,766.148148,656.148148,552.555556,27,POINT (-70.29635 -27.39141)
3601,41846,PARTICULAR SUBVENCIONADO,4,4101,13,9,5.620000,603.500000,638.333333,360.500000,441.500000,6,POINT (-70.22626 -33.45152)
3602,41899,PARTICULAR SUBVENCIONADO,8,8304,426,31,6.465200,846.520000,882.480000,750.800000,682.120000,25,POINT (-72.70880 -37.28291)


In [16]:
gpd_comunas = gpd.read_file(os.path.join("raw_data", "Comunas.zip")).to_crs(pyproj.CRS.from_epsg(5360))
gpd_comunas.to_parquet(os.path.join("data", "comunas.parquet"), engine="pyarrow", compression="snappy")

/home/rocky17/miniconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
